In [1]:
# embedding_layer = Embedding(200, 32, input_length=50) 
# -> Embedding(size of the vocabulary,number of the dimensions for each word vector,length of the input sentence)

#The output of the word embedding is a 2D vector where words are represented in rows, 
#whereas their corresponding dimensions are presented in columns

# If you wish to directly connect your word embedding layer with a densely connected layer,
#you first have to flatten your 2D word embeddings into 1D. 

#Keras can be used to either learn custom words embedding or it can be used to load pretrained word embeddings.

Custom Word Embeddings

In [2]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [3]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [4]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [5]:
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [6]:
unique_words = set(all_words)
print(len(unique_words))

45


In [7]:
vocab_length = len(unique_words) + 5 # adding buffer of 5

In [8]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences)

[[43, 41, 34, 34, 2], [5, 5, 33, 30, 18, 28, 14], [32, 14, 45, 14, 41, 30], [12, 37], [40, 20, 12, 8, 18, 28, 14], [1, 35, 30, 26], [1, 42, 4, 35, 17, 2], [14, 41, 35, 37, 2], [36, 31], [48, 20, 19], [45, 9], [14, 33, 38, 14], [18, 35, 9, 28, 5, 2], [5, 2, 33, 36], [18, 43, 9, 34], [5, 31, 41, 45]]


In [9]:
# The embedding layer expects sentences to be of equal size


word_count = lambda sentence: len(word_tokenize(sentence))

# max with respect to word count
longest_sentence = max(corpus, key=word_count) 

length_long_sentence = len(word_tokenize(longest_sentence))
length_long_sentence

7

In [10]:
# pad_sequences method. 
#The first parameter is the list of encoded sentences of unequal sizes 
#the second parameter is the size of the longest sentence
#the last parameter is padding where you specify post or pre to add padding to sentences.
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[43 41 34 34  2  0  0]
 [ 5  5 33 30 18 28 14]
 [32 14 45 14 41 30  0]
 [12 37  0  0  0  0  0]
 [40 20 12  8 18 28 14]
 [ 1 35 30 26  0  0  0]
 [ 1 42  4 35 17  2  0]
 [14 41 35 37  2  0  0]
 [36 31  0  0  0  0  0]
 [48 20 19  0  0  0  0]
 [45  9  0  0  0  0  0]
 [14 33 38 14  0  0  0]
 [18 35  9 28  5  2  0]
 [ 5  2 33 36  0  0  0]
 [18 43  9 34  0  0  0]
 [ 5 31 41 45  0  0  0]]


In [11]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
#We can see that the first layer has 1000 trainable parameters.
#This is because our vocabulary size is 50 and each word will be presented as a 20 dimensional vector
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

C:\Users\WorkPC\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
16/16 [==============================] - 0s 11ms/step - loss: 0.6948 - acc: 0.4375
Epoch 2/100
16/16 [==============================] - 0s 62us/step - loss: 0.6915 - acc: 0.4375
Epoch 3/100
16/16 [==============================] - 0s 0us/step - loss: 0.6881 - acc: 0.6250
Epoch 4/100
16/16 [==============================] - 0s 125us/step - loss: 0.6848 - acc: 0.6250
Epoch 5/100
16/16 [==============================] - 0s 62us/step - loss: 0.6815 - acc: 0.8125
Epoch 6/100
16/16 [==============================] - 0s 62us/step - loss: 0.6781 - acc: 0.8125
Epoch 7/100
16/16 [==============================] - 0s 62us/step - loss: 0.6748 - acc: 0.8750
Epoch 8/100
16/16 [==============================] - 0s 62us/step - loss: 0.6716 - acc: 0.9375
Epoch 9/100
16/16 [==============================] - 0s 62us/step - loss: 0.6683 - acc: 0.9375
Epoch 10/100
16/16 [==============================] - 0s 62us/step - loss: 0.6650 - acc: 1.0000
Epoch 11/100
16/16 [=============================

Epoch 87/100
16/16 [==============================] - 0s 62us/step - loss: 0.3378 - acc: 1.0000
Epoch 88/100
16/16 [==============================] - 0s 125us/step - loss: 0.3334 - acc: 1.0000
Epoch 89/100
16/16 [==============================] - 0s 62us/step - loss: 0.3291 - acc: 1.0000
Epoch 90/100
16/16 [==============================] - 0s 62us/step - loss: 0.3248 - acc: 1.0000
Epoch 91/100
16/16 [==============================] - 0s 62us/step - loss: 0.3205 - acc: 1.0000
Epoch 92/100
16/16 [==============================] - 0s 62us/step - loss: 0.3162 - acc: 1.0000
Epoch 93/100
16/16 [==============================] - 0s 62us/step - loss: 0.3120 - acc: 1.0000
Epoch 94/100
16/16 [==============================] - 0s 62us/step - loss: 0.3079 - acc: 1.0000
Epoch 95/100
16/16 [==============================] - 0s 62us/step - loss: 0.3037 - acc: 1.0000
Epoch 96/100
16/16 [==============================] - 0s 62us/step - loss: 0.2996 - acc: 1.0000
Epoch 97/100
16/16 [===================

In [14]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


Loading Pretrained Word Embeddings

In [22]:
from keras.preprocessing.text import Tokenizer
import numpy as np

In [17]:
#Before we used one_hot function to convert text to vectors.
#Another approach is to use Tokenizer function from keras.preprocessing.text library.
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

In [18]:
# Adding 1 to store the dimensions for the words for which no pretrained word embeddings exist.
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

44

In [19]:
#Finally, to convert sentences to their numeric counterpart, call the texts_to_sequences function and pass it the whole corpus.
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
print(embedded_sentences)

[[14, 3, 15, 16, 1], [4, 17, 6, 9, 5, 7, 2], [18, 19, 20, 2, 3, 21], [22, 23], [24, 25, 26, 27, 5, 7, 2], [28, 8, 9, 29], [30, 31, 32, 8, 33, 1], [2, 3, 8, 34, 1], [10, 11], [35, 36, 37], [12, 38], [2, 6, 39, 40], [5, 41, 13, 7, 4, 1], [4, 1, 6, 10], [5, 42, 13, 43], [4, 11, 3, 12]]


In [21]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

print(padded_sentences)

[[14  3 15 16  1  0  0]
 [ 4 17  6  9  5  7  2]
 [18 19 20  2  3 21  0]
 [22 23  0  0  0  0  0]
 [24 25 26 27  5  7  2]
 [28  8  9 29  0  0  0]
 [30 31 32  8 33  1  0]
 [ 2  3  8 34  1  0  0]
 [10 11  0  0  0  0  0]
 [35 36 37  0  0  0  0]
 [12 38  0  0  0  0  0]
 [ 2  6 39 40  0  0  0]
 [ 5 41 13  7  4  1  0]
 [ 4  1  6 10  0  0  0]
 [ 5 42 13 43  0  0  0]
 [ 4 11  3 12  0  0  0]]


In [23]:
# The next step is to load the GloVe word embeddings 
# and then create our embedding matrix that contains the words in our corpus and their corresponding values from GloVe embeddings.
embeddings_dictionary = dict() # to store our word embeddings
glove_file = open('glove.6B.100d.txt', encoding="utf8")

In [25]:
#We will create a dictionary that will contain words as keys and the corresponding 100 dimensional vectors as values,
#in the form of an array
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [28]:
len(embeddings_dictionary)

399999

In [31]:
#We will create a two dimensional numpy array of 44 (size of vocabulary) rows and 100 columns. 
#The array will initially contain zeros. The array will be named as embedding_matrix
embedding_matrix = np.zeros((vocab_length, 100))

#The 100 dimensional vectors will be stored at the corresponding index of the word in the embedding_matrix
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [36]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.38251001,  0.14821   ,  0.60601002, ...,  0.058921  ,
         0.091112  ,  0.47283   ],
       [-0.30664   ,  0.16821   ,  0.98510998, ..., -0.38775   ,
         0.36916   ,  0.54521   ],
       ...,
       [ 0.30449   , -0.19628   ,  0.20225   , ..., -0.18385001,
        -0.12432   ,  0.27467999],
       [-0.26703   ,  0.44911   ,  0.55478001, ..., -0.87247002,
         0.83828002,  0.465     ],
       [-0.57547998, -0.043236  , -0.1972    , ..., -0.10507   ,
         0.26554999,  0.32192999]])

In [37]:
embedding_matrix.shape

(44, 100)

In [38]:
#We can pass our pretrained embedding matrix as default weights to the weights parameter.
#And since we are not training the embedding layer, the trainable attribute has been set to False.
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [39]:
# 44 x 100= 4400 params for 44 words and 100 dimensions
# output shape -> for 7 words each sentence and their 100 dimensions
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 7, 100)            4400      
_________________________________________________________________
flatten_2 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 701       
Total params: 5,101
Trainable params: 701
Non-trainable params: 4,400
_________________________________________________________________
None


In [40]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
16/16 [==============================] - 0s 11ms/step - loss: 0.8756 - acc: 0.3125
Epoch 2/100
16/16 [==============================] - 0s 62us/step - loss: 0.8446 - acc: 0.3125
Epoch 3/100
16/16 [==============================] - 0s 62us/step - loss: 0.8150 - acc: 0.3125
Epoch 4/100
16/16 [==============================] - 0s 0us/step - loss: 0.7892 - acc: 0.3125
Epoch 5/100
16/16 [==============================] - 0s 62us/step - loss: 0.7620 - acc: 0.3125
Epoch 6/100
16/16 [==============================] - 0s 62us/step - loss: 0.7360 - acc: 0.3125
Epoch 7/100
16/16 [==============================] - 0s 62us/step - loss: 0.7110 - acc: 0.3750
Epoch 8/100
16/16 [==============================] - 0s 62us/step - loss: 0.6871 - acc: 0.5000
Epoch 9/100
16/16 [==============================] - 0s 62us/step - loss: 0.6640 - acc: 0.6250
Epoch 10/100
16/16 [==============================] - 0s 62us/step - loss: 0.6418 - acc: 0.6875
Epoch 11/100
16/16 [==============================

Epoch 87/100
16/16 [==============================] - 0s 63us/step - loss: 0.0973 - acc: 1.0000
Epoch 88/100
16/16 [==============================] - 0s 60us/step - loss: 0.0959 - acc: 1.0000
Epoch 89/100
16/16 [==============================] - 0s 64us/step - loss: 0.0944 - acc: 1.0000
Epoch 90/100
16/16 [==============================] - 0s 61us/step - loss: 0.0930 - acc: 1.0000
Epoch 91/100
16/16 [==============================] - 0s 64us/step - loss: 0.0917 - acc: 1.0000
Epoch 92/100
16/16 [==============================] - 0s 62us/step - loss: 0.0904 - acc: 1.0000
Epoch 93/100
16/16 [==============================] - 0s 61us/step - loss: 0.0891 - acc: 1.0000
Epoch 94/100
16/16 [==============================] - 0s 62us/step - loss: 0.0878 - acc: 1.0000
Epoch 95/100
16/16 [==============================] - 0s 60us/step - loss: 0.0866 - acc: 1.0000
Epoch 96/100
16/16 [==============================] - 0s 62us/step - loss: 0.0854 - acc: 1.0000
Epoch 97/100
16/16 [====================

In [41]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


Keras functional API

In [42]:
# Most of the advanced deep learning models involving multiple inputs and outputs use the Functional API.
from keras.models import Model
from keras.layers import Input

deep_inputs = Input(shape=(length_long_sentence,)) # pass the length of input vector

embedding = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)(deep_inputs) # line A

flatten = Flatten()(embedding)

hidden = Dense(1, activation='sigmoid')(flatten)

model = Model(inputs=deep_inputs, outputs=hidden)

In [43]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7)                 0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 7, 100)            4400      
_________________________________________________________________
flatten_3 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 701       
Total params: 5,101
Trainable params: 701
Non-trainable params: 4,400
_________________________________________________________________
None


In [44]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)

print('Accuracy: %f' % (accuracy*100))

Epoch 1/100
16/16 [==============================] - 0s 9ms/step - loss: 0.7299 - acc: 0.6250
Epoch 2/100
16/16 [==============================] - 0s 62us/step - loss: 0.7035 - acc: 0.5625
Epoch 3/100
16/16 [==============================] - 0s 62us/step - loss: 0.6748 - acc: 0.5625
Epoch 4/100
16/16 [==============================] - 0s 62us/step - loss: 0.6465 - acc: 0.5625
Epoch 5/100
16/16 [==============================] - 0s 62us/step - loss: 0.6196 - acc: 0.5625
Epoch 6/100
16/16 [==============================] - 0s 62us/step - loss: 0.5944 - acc: 0.6875
Epoch 7/100
16/16 [==============================] - 0s 62us/step - loss: 0.5711 - acc: 0.7500
Epoch 8/100
16/16 [==============================] - 0s 62us/step - loss: 0.5494 - acc: 0.7500
Epoch 9/100
16/16 [==============================] - 0s 0us/step - loss: 0.5293 - acc: 0.8125
Epoch 10/100
16/16 [==============================] - 0s 62us/step - loss: 0.5105 - acc: 0.8750
Epoch 11/100
16/16 [==============================]

Epoch 87/100
16/16 [==============================] - 0s 62us/step - loss: 0.0849 - acc: 1.0000
Epoch 88/100
16/16 [==============================] - 0s 62us/step - loss: 0.0837 - acc: 1.0000
Epoch 89/100
16/16 [==============================] - 0s 62us/step - loss: 0.0825 - acc: 1.0000
Epoch 90/100
16/16 [==============================] - 0s 62us/step - loss: 0.0813 - acc: 1.0000
Epoch 91/100
16/16 [==============================] - 0s 62us/step - loss: 0.0802 - acc: 1.0000
Epoch 92/100
16/16 [==============================] - 0s 62us/step - loss: 0.0791 - acc: 1.0000
Epoch 93/100
16/16 [==============================] - 0s 62us/step - loss: 0.0780 - acc: 1.0000
Epoch 94/100
16/16 [==============================] - 0s 62us/step - loss: 0.0770 - acc: 1.0000
Epoch 95/100
16/16 [==============================] - 0s 62us/step - loss: 0.0760 - acc: 1.0000
Epoch 96/100
16/16 [==============================] - 0s 62us/step - loss: 0.0750 - acc: 1.0000
Epoch 97/100
16/16 [====================

To use text data as input to the deep learning model, we need to convert text to numbers. However unlike machine learning models, passing sparse vector of huge sizes can greately affect deep learning models. Therefore, we need to convert our text to small dense vectors. Word embeddings help us convert text to dense vectors.